In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 6
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 120

puzzle.naming = {
    None: '  ',
    'e': ' _',
    'p': ' +',
    'm': ' -',
}
for i in range(9):
    puzzle.naming[str(i)] = f" {i}"

puzzle.domain_program += f"""
    value(m;p).
    value(N) :- num(N).
    num(0..8).
    num(-N) :- num(N).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
    :- puzzle(C,m), cell(C).
    :- puzzle(C,p), cell(C).
    :- puzzle(C,0), cell(C).
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    
    :- P = #count { D : cell(D), solution(D,p) },
        M = #count { D : cell(D), solution(D,m) },
        |P-M| > 4.
    
    :- 0 { solution(D,m) } 0.
    :- 0 { solution(D,p) } 0.
    :- 5 { solution(D,m) }.
    :- 5 { solution(D,p) }.
    
    :- puzzle(C,V) : solution(C,V), num(V), V > 0.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['ring_around_cell']
puzzle.solution_program += """
    :- cell(C), solution(C,N), num(N),
        P = #count { D : cell(D), ring_around_cell(C,D,1), solution(D,p) },
        M = #count { D : cell(D), ring_around_cell(C,D,1), solution(D,m) },
        N != P - M.
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
....
Stopped after solving timeout..
Solving time: 126.10 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| 1| 1|  |  |  |  |
|  |  |-1|-1|-1|  |
|  |  |-1|  |-1|  |
|  |-1|-3|-2|-1|  |
| 2|  |  |  |  | 1|
| 2|  |  |-1|  | 1|


In [5]:
print(puzzle.pretty_repr_solution())

| 1| 1| 1| 0| 0| 0|
| 0| +|-1|-1|-1| 0|
| 0| -|-1| -|-1| 0|
| 0|-1|-3|-2|-1| 1|
| 2| +| -| -| +| 1|
| 2| +| 0|-1| 0| 1|
